# Summary:
- Based on the dataset including the non-RCT from Dave
- NO SMOTE
- Below networks are tested with below results:
    - SVM (300 epoch)
                         precision    recall  f1-score   support

            0.0               0.80      0.87      0.84      2641
            1.0               0.82      0.73      0.78      2137

            avg / total       0.81      0.81      0.81      4778
            
            Confusion Matrix
                2305  336
                 572 1565

    - DNN + SVM (300 epoch)
                         precision    recall  f1-score   support

            0.0               0.87      0.83      0.85      2641
            1.0               0.80      0.85      0.82      2137

            avg / total       0.84      0.84      0.84      4778

            Confusion Matrix
               2187  454
                325 1812
               
    - Deeper DNN with dropout + SVM (300 epoch)
                         precision    recall  f1-score   support

            0.0               0.86      0.82      0.84      2641
            1.0               0.79      0.84      0.82      2137

            avg / total       0.83      0.83      0.83      4778
    
            Confusion Matrix
                2175  466
                 346 1791
    
    - Inception Batch Normalization + SVM (50 epoch)
                         precision    recall  f1-score   support

            0.0               0.85      0.79      0.82      2641
            1.0               0.76      0.83      0.79      2137

            avg / total       0.81      0.81      0.81      4778

            Confusion Matrix
                2091  550
                 373 1764

In [1]:
import glob
import multiprocessing
import time
import csv
import pickle
import os
import pandas as pd
import shutil
import numpy as np
import warnings
import tqdm
import logging
import mxnet as mx
from bs4 import BeautifulSoup
from urllib.request import urlopen
from os.path import basename
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer, sent_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

%pylab inline
warnings.filterwarnings("ignore")

/home/dc/anaconda3/envs/mxnet/lib/python3.6/site-packages/scipy/sparse/sparsetools.py:20: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


Populating the interactive namespace from numpy and matplotlib


In [2]:
# 2d to 4d conversion
def to4d(img):
    return img.reshape(img.shape[0], 1, 32, 32).astype(np.float32)

In [3]:
def modelMetric(mod, info, nd=True):
    if nd == True: 
        prob = mod.predict(val_iters).asnumpy()
    else:
        prob = mod.predict(val_iters)
    logging.info(info)
    # Check the accuracy from prediction
    val_iters.reset()

    y_batch = []
    for dbatch in val_iters:
        label = dbatch.label[0].asnumpy()
        pad = val_iters.getpad()
        real_size = label.shape[0] - pad
        y_batch.append(label[0:real_size])
    y = np.concatenate(y_batch)

    # get prediction label 
    py = np.argmax(prob, axis=1)
    acc1 = sum(py == y) / len(y)
    logging.info('Test accuracy = %f', acc1)
    cfn_matrix=confusion_matrix(y,py)
    print(classification_report (y,py))
    print(cfn_matrix)

# 1. Data Preparation

In [4]:
p_file = 'RCT_Vectors'
with open(p_file, 'rb') as fin:
    vectors = pickle.load(fin)

In [5]:
p_file = 'RCT_labels'
with open(p_file, 'rb') as fin:
    data_Y = pickle.load(fin)

In [6]:
d2v_X_train, d2v_X_test, d2v_Y_train, d2v_Y_test = train_test_split(vectors, data_Y, test_size=0.25)

In [7]:
X_train_array = np.array(d2v_X_train)
Y_train_array = np.array(d2v_Y_train)
X_test_array = np.array(d2v_X_test)
Y_test_array = np.array(d2v_Y_test)

In [8]:
# double check whether oversampling works
X_train_array.shape, Y_train_array.shape, X_test_array.shape, Y_test_array.shape, len(vectors)

((14333, 1024), (14333,), (4778, 1024), (4778,), 19111)

In [9]:
# creating 4d training/testing dataset
batch_size = 64
train_iters = mx.io.NDArrayIter(to4d(X_train_array), Y_train_array, batch_size, shuffle=True,label_name='svm_label')
val_iters = mx.io.NDArrayIter(to4d(X_test_array), Y_test_array, batch_size,label_name='svm_label')

# 2. Train models
## 2.1 SVM

In [10]:
# Single SVM
data = mx.symbol.Variable('data')
svm  = mx.symbol.FullyConnected(data = data, name='fc', num_hidden=2)

# Here we add the ultimate layer based on L2-SVM objective
ssvm = mx.symbol.SVMOutput(data=svm, name='svm')
# draw network chart
shape = {"data" : (batch_size, 1, 32, 32)}
mx.viz.plot_network(symbol=ssvm, shape=shape)
network_chart=mx.viz.plot_network(symbol=ssvm, shape=shape)
network_chart.format = 'png'
network_chart.render('SVM')

'SVM.png'

<img src='SVM.png'>

In [11]:
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

mod = mx.model.FeedForward(
    ctx = mx.gpu(0),     # use GPU 0 for training, others are same as before
    symbol = ssvm,       # network structure
    num_epoch = 300,     # number of data passes for training 
    learning_rate = 0.01 # learning rate of SGD 
)
mod.fit(
    X=train_iters,       # training data
    eval_data=val_iters, # validation data
    batch_end_callback = mx.callback.Speedometer(batch_size, batch_size) # output progress for each 200 data batches
)


INFO:root:Start training with [gpu(0)]
INFO:root:Epoch[0] Batch [64]	Speed: 62392.39 samples/sec	accuracy=0.792236
INFO:root:Epoch[0] Batch [128]	Speed: 59880.20 samples/sec	accuracy=0.823486
INFO:root:Epoch[0] Batch [192]	Speed: 57760.47 samples/sec	accuracy=0.827393
INFO:root:Epoch[0] Resetting Data Iterator
INFO:root:Epoch[0] Time cost=0.251
INFO:root:Epoch[0] Validation-accuracy=0.811250
INFO:root:Epoch[1] Batch [64]	Speed: 59976.78 samples/sec	accuracy=0.846680
INFO:root:Epoch[1] Batch [128]	Speed: 63838.16 samples/sec	accuracy=0.846680
INFO:root:Epoch[1] Batch [192]	Speed: 65024.77 samples/sec	accuracy=0.840820
INFO:root:Epoch[1] Resetting Data Iterator
INFO:root:Epoch[1] Time cost=0.233
INFO:root:Epoch[1] Validation-accuracy=0.809167
INFO:root:Epoch[2] Batch [64]	Speed: 73296.71 samples/sec	accuracy=0.849609
INFO:root:Epoch[2] Batch [128]	Speed: 69034.27 samples/sec	accuracy=0.848389
INFO:root:Epoch[2] Batch [192]	Speed: 62961.06 samples/sec	accuracy=0.845947
INFO:root:Epoch[2] 

INFO:root:Epoch[22] Time cost=0.198
INFO:root:Epoch[22] Validation-accuracy=0.809583
INFO:root:Epoch[23] Batch [64]	Speed: 69883.74 samples/sec	accuracy=0.855957
INFO:root:Epoch[23] Batch [128]	Speed: 67285.49 samples/sec	accuracy=0.851807
INFO:root:Epoch[23] Batch [192]	Speed: 70459.26 samples/sec	accuracy=0.849854
INFO:root:Epoch[23] Resetting Data Iterator
INFO:root:Epoch[23] Time cost=0.213
INFO:root:Epoch[23] Validation-accuracy=0.809792
INFO:root:Epoch[24] Batch [64]	Speed: 72951.54 samples/sec	accuracy=0.855957
INFO:root:Epoch[24] Batch [128]	Speed: 70505.52 samples/sec	accuracy=0.851807
INFO:root:Epoch[24] Batch [192]	Speed: 64022.53 samples/sec	accuracy=0.849609
INFO:root:Epoch[24] Resetting Data Iterator
INFO:root:Epoch[24] Time cost=0.218
INFO:root:Epoch[24] Validation-accuracy=0.810000
INFO:root:Epoch[25] Batch [64]	Speed: 60158.24 samples/sec	accuracy=0.855469
INFO:root:Epoch[25] Batch [128]	Speed: 63633.86 samples/sec	accuracy=0.851807
INFO:root:Epoch[25] Batch [192]	Spee

INFO:root:Epoch[45] Batch [192]	Speed: 71510.09 samples/sec	accuracy=0.849121
INFO:root:Epoch[45] Resetting Data Iterator
INFO:root:Epoch[45] Time cost=0.209
INFO:root:Epoch[45] Validation-accuracy=0.810833
INFO:root:Epoch[46] Batch [64]	Speed: 72774.78 samples/sec	accuracy=0.854004
INFO:root:Epoch[46] Batch [128]	Speed: 69194.71 samples/sec	accuracy=0.852539
INFO:root:Epoch[46] Batch [192]	Speed: 69167.40 samples/sec	accuracy=0.849121
INFO:root:Epoch[46] Resetting Data Iterator
INFO:root:Epoch[46] Time cost=0.210
INFO:root:Epoch[46] Validation-accuracy=0.810625
INFO:root:Epoch[47] Batch [64]	Speed: 70653.69 samples/sec	accuracy=0.854004
INFO:root:Epoch[47] Batch [128]	Speed: 70653.69 samples/sec	accuracy=0.852783
INFO:root:Epoch[47] Batch [192]	Speed: 71094.90 samples/sec	accuracy=0.848877
INFO:root:Epoch[47] Resetting Data Iterator
INFO:root:Epoch[47] Time cost=0.209
INFO:root:Epoch[47] Validation-accuracy=0.810417
INFO:root:Epoch[48] Batch [64]	Speed: 68885.63 samples/sec	accuracy=0

INFO:root:Epoch[68] Batch [128]	Speed: 73254.52 samples/sec	accuracy=0.853027
INFO:root:Epoch[68] Batch [192]	Speed: 74252.80 samples/sec	accuracy=0.848633
INFO:root:Epoch[68] Resetting Data Iterator
INFO:root:Epoch[68] Time cost=0.208
INFO:root:Epoch[68] Validation-accuracy=0.810208
INFO:root:Epoch[69] Batch [64]	Speed: 76483.47 samples/sec	accuracy=0.853760
INFO:root:Epoch[69] Batch [128]	Speed: 74599.08 samples/sec	accuracy=0.853027
INFO:root:Epoch[69] Batch [192]	Speed: 70946.63 samples/sec	accuracy=0.848633
INFO:root:Epoch[69] Resetting Data Iterator
INFO:root:Epoch[69] Time cost=0.199
INFO:root:Epoch[69] Validation-accuracy=0.810208
INFO:root:Epoch[70] Batch [64]	Speed: 74935.20 samples/sec	accuracy=0.853760
INFO:root:Epoch[70] Batch [128]	Speed: 70754.08 samples/sec	accuracy=0.853027
INFO:root:Epoch[70] Batch [192]	Speed: 71313.28 samples/sec	accuracy=0.848389
INFO:root:Epoch[70] Resetting Data Iterator
INFO:root:Epoch[70] Time cost=0.204
INFO:root:Epoch[70] Validation-accuracy=

INFO:root:Epoch[90] Validation-accuracy=0.810625
INFO:root:Epoch[91] Batch [64]	Speed: 70957.77 samples/sec	accuracy=0.853760
INFO:root:Epoch[91] Batch [128]	Speed: 69210.87 samples/sec	accuracy=0.853027
INFO:root:Epoch[91] Batch [192]	Speed: 70653.69 samples/sec	accuracy=0.848389
INFO:root:Epoch[91] Resetting Data Iterator
INFO:root:Epoch[91] Time cost=0.208
INFO:root:Epoch[91] Validation-accuracy=0.810625
INFO:root:Epoch[92] Batch [64]	Speed: 70244.63 samples/sec	accuracy=0.853760
INFO:root:Epoch[92] Batch [128]	Speed: 67759.19 samples/sec	accuracy=0.853027
INFO:root:Epoch[92] Batch [192]	Speed: 68982.99 samples/sec	accuracy=0.848389
INFO:root:Epoch[92] Resetting Data Iterator
INFO:root:Epoch[92] Time cost=0.212
INFO:root:Epoch[92] Validation-accuracy=0.810625
INFO:root:Epoch[93] Batch [64]	Speed: 73058.57 samples/sec	accuracy=0.853760
INFO:root:Epoch[93] Batch [128]	Speed: 69520.35 samples/sec	accuracy=0.853027
INFO:root:Epoch[93] Batch [192]	Speed: 66039.59 samples/sec	accuracy=0.8

...

INFO:root:Epoch[203] Batch [64]	Speed: 79868.48 samples/sec	accuracy=0.853760
INFO:root:Epoch[203] Batch [128]	Speed: 79863.65 samples/sec	accuracy=0.853027
INFO:root:Epoch[203] Batch [192]	Speed: 75828.56 samples/sec	accuracy=0.848145
INFO:root:Epoch[203] Resetting Data Iterator
INFO:root:Epoch[203] Time cost=0.191
INFO:root:Epoch[203] Validation-accuracy=0.810625
INFO:root:Epoch[204] Batch [64]	Speed: 73520.67 samples/sec	accuracy=0.853760
INFO:root:Epoch[204] Batch [128]	Speed: 72115.24 samples/sec	accuracy=0.853027
INFO:root:Epoch[204] Batch [192]	Speed: 68017.00 samples/sec	accuracy=0.848145
INFO:root:Epoch[204] Resetting Data Iterator
INFO:root:Epoch[204] Time cost=0.207
INFO:root:Epoch[204] Validation-accuracy=0.810625
INFO:root:Epoch[205] Batch [64]	Speed: 72421.67 samples/sec	accuracy=0.853760
INFO:root:Epoch[205] Batch [128]	Speed: 64601.52 samples/sec	accuracy=0.853027
INFO:root:Epoch[205] Batch [192]	Speed: 65364.45 samples/sec	accuracy=0.848145
INFO:root:Epoch[205] Resetti

INFO:root:Epoch[225] Batch [192]	Speed: 64685.19 samples/sec	accuracy=0.848145
INFO:root:Epoch[225] Resetting Data Iterator
INFO:root:Epoch[225] Time cost=0.209
INFO:root:Epoch[225] Validation-accuracy=0.810625
INFO:root:Epoch[226] Batch [64]	Speed: 75757.67 samples/sec	accuracy=0.853760
INFO:root:Epoch[226] Batch [128]	Speed: 70539.10 samples/sec	accuracy=0.853027
INFO:root:Epoch[226] Batch [192]	Speed: 74159.84 samples/sec	accuracy=0.848145
INFO:root:Epoch[226] Resetting Data Iterator
INFO:root:Epoch[226] Time cost=0.201
INFO:root:Epoch[226] Validation-accuracy=0.810625
INFO:root:Epoch[227] Batch [64]	Speed: 80774.22 samples/sec	accuracy=0.853760
INFO:root:Epoch[227] Batch [128]	Speed: 75313.64 samples/sec	accuracy=0.853027
INFO:root:Epoch[227] Batch [192]	Speed: 71446.15 samples/sec	accuracy=0.848145
INFO:root:Epoch[227] Resetting Data Iterator
INFO:root:Epoch[227] Time cost=0.195
INFO:root:Epoch[227] Validation-accuracy=0.810625
INFO:root:Epoch[228] Batch [64]	Speed: 78648.00 sampl

INFO:root:Epoch[247] Time cost=0.194
INFO:root:Epoch[247] Validation-accuracy=0.810625
INFO:root:Epoch[248] Batch [64]	Speed: 69842.26 samples/sec	accuracy=0.853760
INFO:root:Epoch[248] Batch [128]	Speed: 67275.21 samples/sec	accuracy=0.853027
INFO:root:Epoch[248] Batch [192]	Speed: 68876.52 samples/sec	accuracy=0.848145
INFO:root:Epoch[248] Resetting Data Iterator
INFO:root:Epoch[248] Time cost=0.215
INFO:root:Epoch[248] Validation-accuracy=0.810625
INFO:root:Epoch[249] Batch [64]	Speed: 65100.91 samples/sec	accuracy=0.853760
INFO:root:Epoch[249] Batch [128]	Speed: 67030.31 samples/sec	accuracy=0.853027
INFO:root:Epoch[249] Batch [192]	Speed: 75162.73 samples/sec	accuracy=0.848145
INFO:root:Epoch[249] Resetting Data Iterator
INFO:root:Epoch[249] Time cost=0.214
INFO:root:Epoch[249] Validation-accuracy=0.810625
INFO:root:Epoch[250] Batch [64]	Speed: 76214.40 samples/sec	accuracy=0.853760
INFO:root:Epoch[250] Batch [128]	Speed: 73086.85 samples/sec	accuracy=0.853027
INFO:root:Epoch[250]

INFO:root:Epoch[270] Batch [128]	Speed: 68324.56 samples/sec	accuracy=0.853027
INFO:root:Epoch[270] Batch [192]	Speed: 69411.09 samples/sec	accuracy=0.848145
INFO:root:Epoch[270] Resetting Data Iterator
INFO:root:Epoch[270] Time cost=0.216
INFO:root:Epoch[270] Validation-accuracy=0.810625
INFO:root:Epoch[271] Batch [64]	Speed: 73150.34 samples/sec	accuracy=0.853760
INFO:root:Epoch[271] Batch [128]	Speed: 70980.05 samples/sec	accuracy=0.853027
INFO:root:Epoch[271] Batch [192]	Speed: 69677.68 samples/sec	accuracy=0.848145
INFO:root:Epoch[271] Resetting Data Iterator
INFO:root:Epoch[271] Time cost=0.210
INFO:root:Epoch[271] Validation-accuracy=0.810625
INFO:root:Epoch[272] Batch [64]	Speed: 63204.04 samples/sec	accuracy=0.853760
INFO:root:Epoch[272] Batch [128]	Speed: 70791.40 samples/sec	accuracy=0.853027
INFO:root:Epoch[272] Batch [192]	Speed: 74616.90 samples/sec	accuracy=0.848145
INFO:root:Epoch[272] Resetting Data Iterator
INFO:root:Epoch[272] Time cost=0.212
INFO:root:Epoch[272] Val

INFO:root:Epoch[292] Resetting Data Iterator
INFO:root:Epoch[292] Time cost=0.210
INFO:root:Epoch[292] Validation-accuracy=0.810625
INFO:root:Epoch[293] Batch [64]	Speed: 73785.28 samples/sec	accuracy=0.853760
INFO:root:Epoch[293] Batch [128]	Speed: 66289.57 samples/sec	accuracy=0.853027
INFO:root:Epoch[293] Batch [192]	Speed: 71024.65 samples/sec	accuracy=0.848145
INFO:root:Epoch[293] Resetting Data Iterator
INFO:root:Epoch[293] Time cost=0.209
INFO:root:Epoch[293] Validation-accuracy=0.810625
INFO:root:Epoch[294] Batch [64]	Speed: 60186.90 samples/sec	accuracy=0.853760
INFO:root:Epoch[294] Batch [128]	Speed: 78421.47 samples/sec	accuracy=0.853027
INFO:root:Epoch[294] Batch [192]	Speed: 77674.40 samples/sec	accuracy=0.848145
INFO:root:Epoch[294] Resetting Data Iterator
INFO:root:Epoch[294] Time cost=0.208
INFO:root:Epoch[294] Validation-accuracy=0.810625
INFO:root:Epoch[295] Batch [64]	Speed: 80743.85 samples/sec	accuracy=0.853760
INFO:root:Epoch[295] Batch [128]	Speed: 75396.93 sampl

In [12]:
# save a model to mymodel-symbol.json and mymodel-0100.params
prefix = 'SVM'
epoch = 300
mod.save(prefix, epoch)

INFO:root:Saved checkpoint to "SVM-0100.params"


In [13]:
modelMetric(mod, 'Finish predict of one Layer SVM Network...', False)

INFO:root:Finish predict of one Layer SVM Network...
INFO:root:Test accuracy = 0.809962


             precision    recall  f1-score   support

        0.0       0.80      0.87      0.84      2641
        1.0       0.82      0.73      0.78      2137

avg / total       0.81      0.81      0.81      4778

[[2305  336]
 [ 572 1565]]


## 2.2 DNN + SVM

In [14]:
# DNN + SVM 
data = mx.symbol.Variable('data')
data = mx.sym.Flatten(data=data)

fc1  = mx.symbol.FullyConnected(data = data, name='fc1', num_hidden=512)
act1 = mx.symbol.Activation(data = fc1, name='relu1', act_type="relu")
fc2  = mx.symbol.FullyConnected(data = act1, name = 'fc2', num_hidden = 512)
act2 = mx.symbol.Activation(data = fc2, name='relu2', act_type="relu")

fc3  = mx.symbol.FullyConnected(data = act2, name='fc3', num_hidden=2)

dnnsvm = mx.symbol.SVMOutput(data=fc3, name='svm')

# draw network chart
shape = {"data" : (batch_size, 1, 32, 32)}
mx.viz.plot_network(symbol=dnnsvm, shape=shape)
network_chart=mx.viz.plot_network(symbol=dnnsvm, shape=shape)
network_chart.format = 'png'
network_chart.render('DNN + SVM')

'DNN + SVM.png'

<img src='DNN + SVM.png'>

In [15]:
mod = mx.mod.Module(
    context = mx.gpu(0),  # Run on GPU 0
    symbol = dnnsvm,        
    label_names = ['svm_label'],
)
mod.fit(
    train_data=train_iters,
    eval_data=val_iters,  
    batch_end_callback = mx.callback.Speedometer(batch_size, batch_size), 
    num_epoch = 300,       # Train for 100 epochs
    optimizer_params = {
        'learning_rate': 0.001,  # Learning rate
        'momentum': 0.9,       # Momentum for SGD with momentum
        'wd': 0.001,         # Weight decay for regularization
    },
)
print('Accuracy:', mod.score(val_iters, mx.metric.Accuracy())[0][1]*100, '%')

INFO:root:Epoch[0] Batch [64]	Speed: 45448.43 samples/sec	accuracy=0.535817
INFO:root:Epoch[0] Batch [128]	Speed: 47779.19 samples/sec	accuracy=0.557373
INFO:root:Epoch[0] Batch [192]	Speed: 46051.47 samples/sec	accuracy=0.553467
INFO:root:Epoch[0] Train-accuracy=0.534274
INFO:root:Epoch[0] Time cost=0.321
INFO:root:Epoch[0] Validation-accuracy=0.553125
INFO:root:Epoch[1] Batch [64]	Speed: 39602.65 samples/sec	accuracy=0.546154
INFO:root:Epoch[1] Batch [128]	Speed: 43719.35 samples/sec	accuracy=0.567139
INFO:root:Epoch[1] Batch [192]	Speed: 49079.17 samples/sec	accuracy=0.789551
INFO:root:Epoch[1] Train-accuracy=0.814516
INFO:root:Epoch[1] Time cost=0.328
INFO:root:Epoch[1] Validation-accuracy=0.820625
INFO:root:Epoch[2] Batch [64]	Speed: 39632.44 samples/sec	accuracy=0.840625
INFO:root:Epoch[2] Batch [128]	Speed: 49612.80 samples/sec	accuracy=0.846436
INFO:root:Epoch[2] Batch [192]	Speed: 49067.96 samples/sec	accuracy=0.843994
INFO:root:Epoch[2] Train-accuracy=0.847782
INFO:root:Epoch

INFO:root:Epoch[22] Validation-accuracy=0.833125
INFO:root:Epoch[23] Batch [64]	Speed: 48243.56 samples/sec	accuracy=0.996154
INFO:root:Epoch[23] Batch [128]	Speed: 47680.01 samples/sec	accuracy=0.995361
INFO:root:Epoch[23] Batch [192]	Speed: 50960.39 samples/sec	accuracy=0.994141
INFO:root:Epoch[23] Train-accuracy=0.991431
INFO:root:Epoch[23] Time cost=0.295
INFO:root:Epoch[23] Validation-accuracy=0.832083
INFO:root:Epoch[24] Batch [64]	Speed: 47090.07 samples/sec	accuracy=0.996154
INFO:root:Epoch[24] Batch [128]	Speed: 51970.13 samples/sec	accuracy=0.995361
INFO:root:Epoch[24] Batch [192]	Speed: 46991.92 samples/sec	accuracy=0.994629
INFO:root:Epoch[24] Train-accuracy=0.993448
INFO:root:Epoch[24] Time cost=0.302
INFO:root:Epoch[24] Validation-accuracy=0.834375
INFO:root:Epoch[25] Batch [64]	Speed: 53457.89 samples/sec	accuracy=0.996394
INFO:root:Epoch[25] Batch [128]	Speed: 46371.42 samples/sec	accuracy=0.995605
INFO:root:Epoch[25] Batch [192]	Speed: 48141.90 samples/sec	accuracy=0.9

INFO:root:Epoch[45] Train-accuracy=0.999496
INFO:root:Epoch[45] Time cost=0.298
INFO:root:Epoch[45] Validation-accuracy=0.833333
INFO:root:Epoch[46] Batch [64]	Speed: 50140.88 samples/sec	accuracy=0.999760
INFO:root:Epoch[46] Batch [128]	Speed: 43320.82 samples/sec	accuracy=0.999512
INFO:root:Epoch[46] Batch [192]	Speed: 52356.90 samples/sec	accuracy=0.999268
INFO:root:Epoch[46] Train-accuracy=0.999496
INFO:root:Epoch[46] Time cost=0.303
INFO:root:Epoch[46] Validation-accuracy=0.832292
INFO:root:Epoch[47] Batch [64]	Speed: 44888.87 samples/sec	accuracy=0.999760
INFO:root:Epoch[47] Batch [128]	Speed: 47155.47 samples/sec	accuracy=0.999512
INFO:root:Epoch[47] Batch [192]	Speed: 47807.91 samples/sec	accuracy=0.999268
INFO:root:Epoch[47] Train-accuracy=0.999496
INFO:root:Epoch[47] Time cost=0.310
INFO:root:Epoch[47] Validation-accuracy=0.831458
INFO:root:Epoch[48] Batch [64]	Speed: 49612.94 samples/sec	accuracy=0.999760
INFO:root:Epoch[48] Batch [128]	Speed: 44018.89 samples/sec	accuracy=0

INFO:root:Epoch[68] Batch [192]	Speed: 49456.12 samples/sec	accuracy=0.999268
INFO:root:Epoch[68] Train-accuracy=1.000000
INFO:root:Epoch[68] Time cost=0.305
INFO:root:Epoch[68] Validation-accuracy=0.829792
INFO:root:Epoch[69] Batch [64]	Speed: 46664.01 samples/sec	accuracy=0.999760
INFO:root:Epoch[69] Batch [128]	Speed: 50106.22 samples/sec	accuracy=0.999512
INFO:root:Epoch[69] Batch [192]	Speed: 48530.43 samples/sec	accuracy=0.999268
INFO:root:Epoch[69] Train-accuracy=1.000000
INFO:root:Epoch[69] Time cost=0.302
INFO:root:Epoch[69] Validation-accuracy=0.830208
INFO:root:Epoch[70] Batch [64]	Speed: 51820.42 samples/sec	accuracy=0.999760
INFO:root:Epoch[70] Batch [128]	Speed: 50192.74 samples/sec	accuracy=0.999512
INFO:root:Epoch[70] Batch [192]	Speed: 51473.57 samples/sec	accuracy=0.999268
INFO:root:Epoch[70] Train-accuracy=1.000000
INFO:root:Epoch[70] Time cost=0.287
INFO:root:Epoch[70] Validation-accuracy=0.828958
INFO:root:Epoch[71] Batch [64]	Speed: 48147.97 samples/sec	accuracy=0

INFO:root:Epoch[91] Batch [128]	Speed: 48381.87 samples/sec	accuracy=0.999756
INFO:root:Epoch[91] Batch [192]	Speed: 48127.74 samples/sec	accuracy=0.999756
INFO:root:Epoch[91] Train-accuracy=1.000000
INFO:root:Epoch[91] Time cost=0.301
INFO:root:Epoch[91] Validation-accuracy=0.830625
INFO:root:Epoch[92] Batch [64]	Speed: 49557.55 samples/sec	accuracy=1.000000
INFO:root:Epoch[92] Batch [128]	Speed: 51338.21 samples/sec	accuracy=0.999756
INFO:root:Epoch[92] Batch [192]	Speed: 49788.78 samples/sec	accuracy=0.999756
INFO:root:Epoch[92] Train-accuracy=1.000000
INFO:root:Epoch[92] Time cost=0.291
INFO:root:Epoch[92] Validation-accuracy=0.829792
INFO:root:Epoch[93] Batch [64]	Speed: 46783.33 samples/sec	accuracy=1.000000
INFO:root:Epoch[93] Batch [128]	Speed: 46704.86 samples/sec	accuracy=0.999756
INFO:root:Epoch[93] Batch [192]	Speed: 47286.69 samples/sec	accuracy=0.999756
INFO:root:Epoch[93] Train-accuracy=1.000000
INFO:root:Epoch[93] Time cost=0.311
INFO:root:Epoch[93] Validation-accuracy=

...

INFO:root:Epoch[225] Train-accuracy=1.000000
INFO:root:Epoch[225] Time cost=0.292
INFO:root:Epoch[225] Validation-accuracy=0.837292
INFO:root:Epoch[226] Batch [64]	Speed: 41913.26 samples/sec	accuracy=1.000000
INFO:root:Epoch[226] Batch [128]	Speed: 48067.95 samples/sec	accuracy=1.000000
INFO:root:Epoch[226] Batch [192]	Speed: 49762.54 samples/sec	accuracy=1.000000
INFO:root:Epoch[226] Train-accuracy=1.000000
INFO:root:Epoch[226] Time cost=0.310
INFO:root:Epoch[226] Validation-accuracy=0.837292
INFO:root:Epoch[227] Batch [64]	Speed: 47771.36 samples/sec	accuracy=1.000000
INFO:root:Epoch[227] Batch [128]	Speed: 49944.96 samples/sec	accuracy=1.000000
INFO:root:Epoch[227] Batch [192]	Speed: 54420.58 samples/sec	accuracy=1.000000
INFO:root:Epoch[227] Train-accuracy=1.000000
INFO:root:Epoch[227] Time cost=0.288
INFO:root:Epoch[227] Validation-accuracy=0.835208
INFO:root:Epoch[228] Batch [64]	Speed: 54952.37 samples/sec	accuracy=1.000000
INFO:root:Epoch[228] Batch [128]	Speed: 50233.68 sampl

INFO:root:Epoch[248] Batch [64]	Speed: 46932.35 samples/sec	accuracy=1.000000
INFO:root:Epoch[248] Batch [128]	Speed: 46365.41 samples/sec	accuracy=1.000000
INFO:root:Epoch[248] Batch [192]	Speed: 51759.69 samples/sec	accuracy=1.000000
INFO:root:Epoch[248] Train-accuracy=1.000000
INFO:root:Epoch[248] Time cost=0.304
INFO:root:Epoch[248] Validation-accuracy=0.834583
INFO:root:Epoch[249] Batch [64]	Speed: 47494.94 samples/sec	accuracy=1.000000
INFO:root:Epoch[249] Batch [128]	Speed: 50758.49 samples/sec	accuracy=1.000000
INFO:root:Epoch[249] Batch [192]	Speed: 50936.82 samples/sec	accuracy=1.000000
INFO:root:Epoch[249] Train-accuracy=1.000000
INFO:root:Epoch[249] Time cost=0.293
INFO:root:Epoch[249] Validation-accuracy=0.834583
INFO:root:Epoch[250] Batch [64]	Speed: 46013.23 samples/sec	accuracy=1.000000
INFO:root:Epoch[250] Batch [128]	Speed: 46301.68 samples/sec	accuracy=1.000000
INFO:root:Epoch[250] Batch [192]	Speed: 47716.16 samples/sec	accuracy=1.000000
INFO:root:Epoch[250] Train-a

INFO:root:Epoch[270] Batch [192]	Speed: 50172.95 samples/sec	accuracy=1.000000
INFO:root:Epoch[270] Train-accuracy=1.000000
INFO:root:Epoch[270] Time cost=0.306
INFO:root:Epoch[270] Validation-accuracy=0.833333
INFO:root:Epoch[271] Batch [64]	Speed: 51130.87 samples/sec	accuracy=1.000000
INFO:root:Epoch[271] Batch [128]	Speed: 51603.29 samples/sec	accuracy=1.000000
INFO:root:Epoch[271] Batch [192]	Speed: 48255.48 samples/sec	accuracy=1.000000
INFO:root:Epoch[271] Train-accuracy=1.000000
INFO:root:Epoch[271] Time cost=0.290
INFO:root:Epoch[271] Validation-accuracy=0.834375
INFO:root:Epoch[272] Batch [64]	Speed: 45531.06 samples/sec	accuracy=1.000000
INFO:root:Epoch[272] Batch [128]	Speed: 50092.19 samples/sec	accuracy=1.000000
INFO:root:Epoch[272] Batch [192]	Speed: 51918.30 samples/sec	accuracy=1.000000
INFO:root:Epoch[272] Train-accuracy=1.000000
INFO:root:Epoch[272] Time cost=0.300
INFO:root:Epoch[272] Validation-accuracy=0.835000
INFO:root:Epoch[273] Batch [64]	Speed: 45870.84 sampl

INFO:root:Epoch[292] Time cost=0.294
INFO:root:Epoch[292] Validation-accuracy=0.837500
INFO:root:Epoch[293] Batch [64]	Speed: 55020.13 samples/sec	accuracy=1.000000
INFO:root:Epoch[293] Batch [128]	Speed: 51279.21 samples/sec	accuracy=1.000000
INFO:root:Epoch[293] Batch [192]	Speed: 48190.65 samples/sec	accuracy=1.000000
INFO:root:Epoch[293] Train-accuracy=1.000000
INFO:root:Epoch[293] Time cost=0.285
INFO:root:Epoch[293] Validation-accuracy=0.836042
INFO:root:Epoch[294] Batch [64]	Speed: 47968.63 samples/sec	accuracy=1.000000
INFO:root:Epoch[294] Batch [128]	Speed: 47006.32 samples/sec	accuracy=1.000000
INFO:root:Epoch[294] Batch [192]	Speed: 50273.37 samples/sec	accuracy=1.000000
INFO:root:Epoch[294] Train-accuracy=1.000000
INFO:root:Epoch[294] Time cost=0.301
INFO:root:Epoch[294] Validation-accuracy=0.837292
INFO:root:Epoch[295] Batch [64]	Speed: 50437.20 samples/sec	accuracy=1.000000
INFO:root:Epoch[295] Batch [128]	Speed: 49377.23 samples/sec	accuracy=1.000000
INFO:root:Epoch[295]

Accuracy: 83.70833333333333 %


In [16]:
# save a model to mymodel-symbol.json and mymodel-0100.params
prefix = 'DNN_SVM'
epoch = 300
mod.save_checkpoint(prefix, epoch)

INFO:root:Saved checkpoint to "DNN_SVM-0300.params"


In [17]:
modelMetric(mod, 'Finish predict of DNN+SVM...')

INFO:root:Finish predict of DNN+SVM...
INFO:root:Test accuracy = 0.836961


             precision    recall  f1-score   support

        0.0       0.87      0.83      0.85      2641
        1.0       0.80      0.85      0.82      2137

avg / total       0.84      0.84      0.84      4778

[[2187  454]
 [ 325 1812]]


## 2.3 Deeper DNN with dropout + SVM

In [18]:
# Deeper DNN with dropout + SVM
data = mx.symbol.Variable('data')
data = mx.sym.Flatten(data=data)

fc1  = mx.symbol.FullyConnected(data = data, name='fc1', num_hidden=1024)
act1 = mx.symbol.Activation(data = fc1, name='relu1', act_type="relu")
fc2  = mx.symbol.FullyConnected(data = act1, name = 'fc2', num_hidden = 1024)
act2 = mx.symbol.Activation(data = fc2, name='relu2', act_type="relu")
drop1 = mx.sym.Dropout(data=act2, p=0.5)

fc3  = mx.symbol.FullyConnected(data = drop1, name = 'fc3', num_hidden = 512)
act3 = mx.symbol.Activation(data = fc3, name='relu3', act_type="relu")
fc4  = mx.symbol.FullyConnected(data = act3, name = 'fc4', num_hidden = 512)
act4 = mx.symbol.Activation(data = fc4, name='relu4', act_type="relu")

drop2 = mx.sym.Dropout(data=act4, p=0.5)

fc5  = mx.symbol.FullyConnected(data = drop2, name='fc5', num_hidden=2)

ddnn = mx.symbol.SVMOutput(data=fc5, name='svm')

# draw network chart
shape = {"data" : (batch_size, 1, 32, 32)}
mx.viz.plot_network(symbol=ddnn, shape=shape)
network_chart=mx.viz.plot_network(symbol=ddnn, shape=shape)
network_chart.format = 'png'
network_chart.render('DDNN + SVM')

'DDNN + SVM.png'

<img src='DDNN + SVM.png'>

In [19]:
mod = mx.mod.Module(
    context = mx.gpu(0),  # Run on GPU 0
    symbol = ddnn,         # Use the network we just defined
    label_names = ['svm_label'],
)
mod.fit(
    train_data=train_iters,
    eval_data=val_iters,  
    batch_end_callback = mx.callback.Speedometer(batch_size, batch_size),  
    num_epoch = 300,       # Train for 100 epochs
    optimizer_params = {
        'learning_rate': 0.001,  # Learning rate
        'momentum': 0.9,       # Momentum for SGD with momentum
        'wd': 0.00001,         # Weight decay for regularization
    },
)

print('Accuracy:', mod.score(val_iters, mx.metric.Accuracy())[0][1]*100, '%')

INFO:root:Epoch[0] Batch [64]	Speed: 33499.86 samples/sec	accuracy=0.541106
INFO:root:Epoch[0] Batch [128]	Speed: 30272.58 samples/sec	accuracy=0.557373
INFO:root:Epoch[0] Batch [192]	Speed: 31268.93 samples/sec	accuracy=0.553467
INFO:root:Epoch[0] Train-accuracy=0.534274
INFO:root:Epoch[0] Time cost=0.484
INFO:root:Epoch[0] Validation-accuracy=0.553125
INFO:root:Epoch[1] Batch [64]	Speed: 33323.06 samples/sec	accuracy=0.546154
INFO:root:Epoch[1] Batch [128]	Speed: 33400.48 samples/sec	accuracy=0.557373
INFO:root:Epoch[1] Batch [192]	Speed: 34006.75 samples/sec	accuracy=0.553467
INFO:root:Epoch[1] Train-accuracy=0.534274
INFO:root:Epoch[1] Time cost=0.435
INFO:root:Epoch[1] Validation-accuracy=0.553125
INFO:root:Epoch[2] Batch [64]	Speed: 32804.29 samples/sec	accuracy=0.546154
INFO:root:Epoch[2] Batch [128]	Speed: 33273.49 samples/sec	accuracy=0.557373
INFO:root:Epoch[2] Batch [192]	Speed: 33946.00 samples/sec	accuracy=0.553467
INFO:root:Epoch[2] Train-accuracy=0.534274
INFO:root:Epoch

INFO:root:Epoch[22] Validation-accuracy=0.553125
INFO:root:Epoch[23] Batch [64]	Speed: 37460.82 samples/sec	accuracy=0.546154
INFO:root:Epoch[23] Batch [128]	Speed: 36076.78 samples/sec	accuracy=0.557373
INFO:root:Epoch[23] Batch [192]	Speed: 35773.36 samples/sec	accuracy=0.553467
INFO:root:Epoch[23] Train-accuracy=0.534274
INFO:root:Epoch[23] Time cost=0.405
INFO:root:Epoch[23] Validation-accuracy=0.553125
INFO:root:Epoch[24] Batch [64]	Speed: 31943.83 samples/sec	accuracy=0.546154
INFO:root:Epoch[24] Batch [128]	Speed: 34816.88 samples/sec	accuracy=0.557373
INFO:root:Epoch[24] Batch [192]	Speed: 34269.81 samples/sec	accuracy=0.553467
INFO:root:Epoch[24] Train-accuracy=0.534274
INFO:root:Epoch[24] Time cost=0.427
INFO:root:Epoch[24] Validation-accuracy=0.553125
INFO:root:Epoch[25] Batch [64]	Speed: 31047.19 samples/sec	accuracy=0.546154
INFO:root:Epoch[25] Batch [128]	Speed: 34186.21 samples/sec	accuracy=0.557373
INFO:root:Epoch[25] Batch [192]	Speed: 37119.34 samples/sec	accuracy=0.5

INFO:root:Epoch[45] Train-accuracy=0.987399
INFO:root:Epoch[45] Time cost=0.427
INFO:root:Epoch[45] Validation-accuracy=0.839583
INFO:root:Epoch[46] Batch [64]	Speed: 34941.09 samples/sec	accuracy=0.992788
INFO:root:Epoch[46] Batch [128]	Speed: 36963.52 samples/sec	accuracy=0.993408
INFO:root:Epoch[46] Batch [192]	Speed: 32278.43 samples/sec	accuracy=0.990234
INFO:root:Epoch[46] Train-accuracy=0.987399
INFO:root:Epoch[46] Time cost=0.416
INFO:root:Epoch[46] Validation-accuracy=0.840000
INFO:root:Epoch[47] Batch [64]	Speed: 32163.79 samples/sec	accuracy=0.992788
INFO:root:Epoch[47] Batch [128]	Speed: 35467.90 samples/sec	accuracy=0.993408
INFO:root:Epoch[47] Batch [192]	Speed: 36162.74 samples/sec	accuracy=0.990234
INFO:root:Epoch[47] Train-accuracy=0.987399
INFO:root:Epoch[47] Time cost=0.416
INFO:root:Epoch[47] Validation-accuracy=0.840625
INFO:root:Epoch[48] Batch [64]	Speed: 31532.94 samples/sec	accuracy=0.992788
INFO:root:Epoch[48] Batch [128]	Speed: 36525.48 samples/sec	accuracy=0

INFO:root:Epoch[68] Batch [192]	Speed: 37316.15 samples/sec	accuracy=0.991943
INFO:root:Epoch[68] Train-accuracy=0.989919
INFO:root:Epoch[68] Time cost=0.408
INFO:root:Epoch[68] Validation-accuracy=0.839583
INFO:root:Epoch[69] Batch [64]	Speed: 31408.59 samples/sec	accuracy=0.993750
INFO:root:Epoch[69] Batch [128]	Speed: 37580.62 samples/sec	accuracy=0.994629
INFO:root:Epoch[69] Batch [192]	Speed: 37027.81 samples/sec	accuracy=0.991943
INFO:root:Epoch[69] Train-accuracy=0.989919
INFO:root:Epoch[69] Time cost=0.411
INFO:root:Epoch[69] Validation-accuracy=0.840417
INFO:root:Epoch[70] Batch [64]	Speed: 31285.16 samples/sec	accuracy=0.993750
INFO:root:Epoch[70] Batch [128]	Speed: 36861.77 samples/sec	accuracy=0.994629
INFO:root:Epoch[70] Batch [192]	Speed: 35715.65 samples/sec	accuracy=0.991943
INFO:root:Epoch[70] Train-accuracy=0.989919
INFO:root:Epoch[70] Time cost=0.428
INFO:root:Epoch[70] Validation-accuracy=0.840208
INFO:root:Epoch[71] Batch [64]	Speed: 32555.94 samples/sec	accuracy=0

INFO:root:Epoch[91] Batch [128]	Speed: 36322.77 samples/sec	accuracy=0.995605
INFO:root:Epoch[91] Batch [192]	Speed: 36584.75 samples/sec	accuracy=0.994629
INFO:root:Epoch[91] Train-accuracy=0.991935
INFO:root:Epoch[91] Time cost=0.417
INFO:root:Epoch[91] Validation-accuracy=0.835625
INFO:root:Epoch[92] Batch [64]	Speed: 31435.66 samples/sec	accuracy=0.995673
INFO:root:Epoch[92] Batch [128]	Speed: 35999.44 samples/sec	accuracy=0.995605
INFO:root:Epoch[92] Batch [192]	Speed: 35835.07 samples/sec	accuracy=0.994873
INFO:root:Epoch[92] Train-accuracy=0.992440
INFO:root:Epoch[92] Time cost=0.420
INFO:root:Epoch[92] Validation-accuracy=0.835000
INFO:root:Epoch[93] Batch [64]	Speed: 31338.00 samples/sec	accuracy=0.995913
INFO:root:Epoch[93] Batch [128]	Speed: 36583.27 samples/sec	accuracy=0.995605
INFO:root:Epoch[93] Batch [192]	Speed: 35382.65 samples/sec	accuracy=0.995117
INFO:root:Epoch[93] Train-accuracy=0.992440
INFO:root:Epoch[93] Time cost=0.423
INFO:root:Epoch[93] Validation-accuracy=

...

INFO:root:Epoch[225] Train-accuracy=1.000000
INFO:root:Epoch[225] Time cost=0.410
INFO:root:Epoch[225] Validation-accuracy=0.831250
INFO:root:Epoch[226] Batch [64]	Speed: 31769.80 samples/sec	accuracy=1.000000
INFO:root:Epoch[226] Batch [128]	Speed: 35687.83 samples/sec	accuracy=1.000000
INFO:root:Epoch[226] Batch [192]	Speed: 35818.11 samples/sec	accuracy=1.000000
INFO:root:Epoch[226] Train-accuracy=1.000000
INFO:root:Epoch[226] Time cost=0.422
INFO:root:Epoch[226] Validation-accuracy=0.830208
INFO:root:Epoch[227] Batch [64]	Speed: 31432.78 samples/sec	accuracy=1.000000
INFO:root:Epoch[227] Batch [128]	Speed: 34504.10 samples/sec	accuracy=1.000000
INFO:root:Epoch[227] Batch [192]	Speed: 36055.05 samples/sec	accuracy=1.000000
INFO:root:Epoch[227] Train-accuracy=1.000000
INFO:root:Epoch[227] Time cost=0.425
INFO:root:Epoch[227] Validation-accuracy=0.829583
INFO:root:Epoch[228] Batch [64]	Speed: 32115.33 samples/sec	accuracy=1.000000
INFO:root:Epoch[228] Batch [128]	Speed: 37253.68 sampl

INFO:root:Epoch[248] Batch [64]	Speed: 31841.11 samples/sec	accuracy=1.000000
INFO:root:Epoch[248] Batch [128]	Speed: 36723.56 samples/sec	accuracy=1.000000
INFO:root:Epoch[248] Batch [192]	Speed: 36622.96 samples/sec	accuracy=1.000000
INFO:root:Epoch[248] Train-accuracy=1.000000
INFO:root:Epoch[248] Time cost=0.412
INFO:root:Epoch[248] Validation-accuracy=0.829583
INFO:root:Epoch[249] Batch [64]	Speed: 34782.20 samples/sec	accuracy=1.000000
INFO:root:Epoch[249] Batch [128]	Speed: 35090.68 samples/sec	accuracy=1.000000
INFO:root:Epoch[249] Batch [192]	Speed: 35963.65 samples/sec	accuracy=1.000000
INFO:root:Epoch[249] Train-accuracy=1.000000
INFO:root:Epoch[249] Time cost=0.408
INFO:root:Epoch[249] Validation-accuracy=0.829375
INFO:root:Epoch[250] Batch [64]	Speed: 31555.06 samples/sec	accuracy=1.000000
INFO:root:Epoch[250] Batch [128]	Speed: 35091.68 samples/sec	accuracy=1.000000
INFO:root:Epoch[250] Batch [192]	Speed: 37952.89 samples/sec	accuracy=1.000000
INFO:root:Epoch[250] Train-a

INFO:root:Epoch[270] Batch [192]	Speed: 35830.28 samples/sec	accuracy=1.000000
INFO:root:Epoch[270] Train-accuracy=1.000000
INFO:root:Epoch[270] Time cost=0.421
INFO:root:Epoch[270] Validation-accuracy=0.830208
INFO:root:Epoch[271] Batch [64]	Speed: 31478.57 samples/sec	accuracy=1.000000
INFO:root:Epoch[271] Batch [128]	Speed: 36217.94 samples/sec	accuracy=1.000000
INFO:root:Epoch[271] Batch [192]	Speed: 36374.99 samples/sec	accuracy=1.000000
INFO:root:Epoch[271] Train-accuracy=1.000000
INFO:root:Epoch[271] Time cost=0.417
INFO:root:Epoch[271] Validation-accuracy=0.829792
INFO:root:Epoch[272] Batch [64]	Speed: 33398.01 samples/sec	accuracy=1.000000
INFO:root:Epoch[272] Batch [128]	Speed: 36060.72 samples/sec	accuracy=1.000000
INFO:root:Epoch[272] Batch [192]	Speed: 36065.26 samples/sec	accuracy=1.000000
INFO:root:Epoch[272] Train-accuracy=1.000000
INFO:root:Epoch[272] Time cost=0.412
INFO:root:Epoch[272] Validation-accuracy=0.829375
INFO:root:Epoch[273] Batch [64]	Speed: 36332.14 sampl

INFO:root:Epoch[292] Time cost=0.419
INFO:root:Epoch[292] Validation-accuracy=0.829792
INFO:root:Epoch[293] Batch [64]	Speed: 35395.99 samples/sec	accuracy=1.000000
INFO:root:Epoch[293] Batch [128]	Speed: 30661.80 samples/sec	accuracy=1.000000
INFO:root:Epoch[293] Batch [192]	Speed: 35648.28 samples/sec	accuracy=1.000000
INFO:root:Epoch[293] Train-accuracy=1.000000
INFO:root:Epoch[293] Time cost=0.425
INFO:root:Epoch[293] Validation-accuracy=0.829167
INFO:root:Epoch[294] Batch [64]	Speed: 35565.92 samples/sec	accuracy=1.000000
INFO:root:Epoch[294] Batch [128]	Speed: 36697.75 samples/sec	accuracy=1.000000
INFO:root:Epoch[294] Batch [192]	Speed: 36369.67 samples/sec	accuracy=1.000000
INFO:root:Epoch[294] Train-accuracy=1.000000
INFO:root:Epoch[294] Time cost=0.402
INFO:root:Epoch[294] Validation-accuracy=0.830000
INFO:root:Epoch[295] Batch [64]	Speed: 32068.39 samples/sec	accuracy=1.000000
INFO:root:Epoch[295] Batch [128]	Speed: 36718.22 samples/sec	accuracy=1.000000
INFO:root:Epoch[295]

Accuracy: 83.0 %


In [20]:
# save a model to mymodel-symbol.json and mymodel-0100.params
prefix = 'DDNN_SVM'
epoch = 300
mod.save_checkpoint(prefix, epoch)

INFO:root:Saved checkpoint to "DDNN_SVM-0300.params"


In [21]:
modelMetric(mod, 'Finish predict of DDNN + SVM...')

INFO:root:Finish predict of DDNN + SVM...
INFO:root:Test accuracy = 0.830054


             precision    recall  f1-score   support

        0.0       0.86      0.82      0.84      2641
        1.0       0.79      0.84      0.82      2137

avg / total       0.83      0.83      0.83      4778

[[2175  466]
 [ 346 1791]]


## 2.4 Inception Batach Normalization

In [22]:
# Reference: www.mxnet.io
# Basic Conv + BN + ReLU factory
def ConvFactory(data, num_filter, kernel, stride=(1,1), pad=(0, 0), act_type="relu"):
    conv = mx.symbol.Convolution(data=data, workspace=256,
                                 num_filter=num_filter, kernel=kernel, stride=stride, pad=pad)
    bn = mx.symbol.BatchNorm(data=conv)
    act = mx.symbol.Activation(data = bn, act_type=act_type)
    return act

# A Simple Downsampling Factory
def DownsampleFactory(data, ch_3x3):
    conv = ConvFactory(data=data, kernel=(3, 3), stride=(2, 2), num_filter=ch_3x3, pad=(1, 1))
    pool = mx.symbol.Pooling(data=data, kernel=(3, 3), stride=(2, 2),pad=(1,1), pool_type='max')
    concat = mx.symbol.Concat(*[conv, pool])
    return concat

# A Simple module
def SimpleFactory(data, ch_1x1, ch_3x3):
    conv1x1 = ConvFactory(data=data, kernel=(1, 1), pad=(0, 0), num_filter=ch_1x1)
    conv3x3 = ConvFactory(data=data, kernel=(3, 3), pad=(1, 1), num_filter=ch_3x3)
    concat = mx.symbol.Concat(*[conv1x1, conv3x3])
    return concat

def InceptionFactory(data, num_1x1, num_3x3red, num_3x3, num_d5x5red, num_d5x5, pool, proj, name):
    c1x1 = ConvFactoryB(data=data, num_filter=num_1x1, kernel=(1, 1), name=('%s_1x1' % name))
    c3x3r = ConvFactoryB(data=data, num_filter=num_3x3red, kernel=(1, 1), name=('%s_3x3' % name), suffix='_reduce')
    c3x3 = ConvFactoryB(data=c3x3r, num_filter=num_3x3, kernel=(3, 3), pad=(1, 1), name=('%s_3x3' % name))
    cd5x5r = ConvFactoryB(data=data, num_filter=num_d5x5red, kernel=(1, 1), name=('%s_double_3x3' % name), suffix='_reduce')
    cd5x5 = ConvFactoryB(data=cd5x5r, num_filter=num_d5x5, kernel=(5, 5), pad=(2, 2), name=('%s_double_3x3_1' % name))
    pooling = mx.symbol.Pooling(data=data, kernel=(3, 3), stride=(1, 1), pad=(1, 1), pool_type=pool, name=('%s_pool_%s_pool' % (pool, name)))
    cproj = ConvFactoryB(data=pooling, num_filter=proj, kernel=(1, 1), name=('%s_proj' %  name))
    concat = mx.symbol.Concat(*[c1x1, c3x3, cd5x5, cproj], name='ch_concat_%s_chconcat' % name)
    return concat

In [23]:
# Inception Batch Normalization Network
#https://arxiv.org/pdf/1502.03167.pdf
data = mx.sym.Variable('data')

conv1 = ConvFactory(data=data, kernel=(3,3), pad=(1,1), num_filter=96, act_type="relu")
in3a = SimpleFactory(conv1, 32, 32)
in3b = SimpleFactory(in3a, 32, 48)
in3c = DownsampleFactory(in3b, 80)
in4a = SimpleFactory(in3c, 112, 48)
in4b = SimpleFactory(in4a, 96, 64)
in4c = SimpleFactory(in4b, 80, 80)
in4d = SimpleFactory(in4c, 48, 96)
in4e = DownsampleFactory(in4d, 96)
in5a = SimpleFactory(in4e, 176, 160)
in5b = SimpleFactory(in5a, 176, 160)
pool = mx.symbol.Pooling(data=in5b, pool_type="avg", kernel=(7,7), name="global_pool")
flatten = mx.symbol.Flatten(data=pool, name="flatten1")
fc = mx.symbol.FullyConnected(data=flatten, num_hidden=62, name="fc1")
ibn = mx.symbol.SVMOutput(data=fc, name='svm')

# draw network chart
shape = {"data" : (batch_size, 1, 32, 32)}
mx.viz.plot_network(symbol=ibn, shape=shape)
network_chart=mx.viz.plot_network(symbol=ibn, shape=shape)
network_chart.format = 'png'
network_chart.render('Inception Batch Normalization')

'Inception Batch Normalization.png'

<img src = 'Inception Batch Normalization.png'>

In [24]:
mod = mx.mod.Module(
    context = mx.gpu(0),  # Run on GPU 0
    symbol = ibn,         # Use the network we just defined
    label_names = ['svm_label'],
)
mod.fit(
    train_data=train_iters,
    eval_data=val_iters,  
    batch_end_callback = mx.callback.Speedometer(batch_size, batch_size),  
    num_epoch = 50,       # Train for 100 epochs
    optimizer_params = {
        'learning_rate': 0.001,  # Learning rate
        'momentum': 0.9,       # Momentum for SGD with momentum
        'wd': 0.00001,         # Weight decay for regularization
    },
)

print('Accuracy:', mod.score(val_iters, mx.metric.Accuracy())[0][1]*100, '%')

INFO:root:Epoch[0] Batch [64]	Speed: 1188.10 samples/sec	accuracy=0.589663
INFO:root:Epoch[0] Batch [128]	Speed: 1187.39 samples/sec	accuracy=0.731445
INFO:root:Epoch[0] Batch [192]	Speed: 1184.85 samples/sec	accuracy=0.767578
INFO:root:Epoch[0] Train-accuracy=0.763105
INFO:root:Epoch[0] Time cost=12.112
INFO:root:Epoch[0] Validation-accuracy=0.576042
INFO:root:Epoch[1] Batch [64]	Speed: 1194.92 samples/sec	accuracy=0.784375
INFO:root:Epoch[1] Batch [128]	Speed: 1191.24 samples/sec	accuracy=0.812744
INFO:root:Epoch[1] Batch [192]	Speed: 1192.64 samples/sec	accuracy=0.829346
INFO:root:Epoch[1] Train-accuracy=0.814516
INFO:root:Epoch[1] Time cost=12.001
INFO:root:Epoch[1] Validation-accuracy=0.665625
INFO:root:Epoch[2] Batch [64]	Speed: 1190.71 samples/sec	accuracy=0.831971
INFO:root:Epoch[2] Batch [128]	Speed: 1190.32 samples/sec	accuracy=0.848389
INFO:root:Epoch[2] Batch [192]	Speed: 1189.55 samples/sec	accuracy=0.866455
INFO:root:Epoch[2] Train-accuracy=0.845766
INFO:root:Epoch[2] Tim

INFO:root:Epoch[23] Batch [64]	Speed: 1179.13 samples/sec	accuracy=0.995192
INFO:root:Epoch[23] Batch [128]	Speed: 1183.29 samples/sec	accuracy=0.996338
INFO:root:Epoch[23] Batch [192]	Speed: 1178.54 samples/sec	accuracy=0.993896
INFO:root:Epoch[23] Train-accuracy=0.993448
INFO:root:Epoch[23] Time cost=12.113
INFO:root:Epoch[23] Validation-accuracy=0.788750
INFO:root:Epoch[24] Batch [64]	Speed: 1182.31 samples/sec	accuracy=0.995913
INFO:root:Epoch[24] Batch [128]	Speed: 1176.34 samples/sec	accuracy=0.997070
INFO:root:Epoch[24] Batch [192]	Speed: 1174.89 samples/sec	accuracy=0.997070
INFO:root:Epoch[24] Train-accuracy=0.996472
INFO:root:Epoch[24] Time cost=12.153
INFO:root:Epoch[24] Validation-accuracy=0.804167
INFO:root:Epoch[25] Batch [64]	Speed: 1178.89 samples/sec	accuracy=0.997115
INFO:root:Epoch[25] Batch [128]	Speed: 1181.24 samples/sec	accuracy=0.998047
INFO:root:Epoch[25] Batch [192]	Speed: 1184.17 samples/sec	accuracy=0.996582
INFO:root:Epoch[25] Train-accuracy=0.997984
INFO:r

INFO:root:Epoch[45] Time cost=12.090
INFO:root:Epoch[45] Validation-accuracy=0.807083
INFO:root:Epoch[46] Batch [64]	Speed: 1183.49 samples/sec	accuracy=1.000000
INFO:root:Epoch[46] Batch [128]	Speed: 1184.18 samples/sec	accuracy=1.000000
INFO:root:Epoch[46] Batch [192]	Speed: 1183.31 samples/sec	accuracy=1.000000
INFO:root:Epoch[46] Train-accuracy=1.000000
INFO:root:Epoch[46] Time cost=12.096
INFO:root:Epoch[46] Validation-accuracy=0.807083
INFO:root:Epoch[47] Batch [64]	Speed: 1184.06 samples/sec	accuracy=1.000000
INFO:root:Epoch[47] Batch [128]	Speed: 1181.12 samples/sec	accuracy=1.000000
INFO:root:Epoch[47] Batch [192]	Speed: 1178.84 samples/sec	accuracy=1.000000
INFO:root:Epoch[47] Train-accuracy=1.000000
INFO:root:Epoch[47] Time cost=12.097
INFO:root:Epoch[47] Validation-accuracy=0.807083
INFO:root:Epoch[48] Batch [64]	Speed: 1181.63 samples/sec	accuracy=1.000000
INFO:root:Epoch[48] Batch [128]	Speed: 1178.85 samples/sec	accuracy=1.000000
INFO:root:Epoch[48] Batch [192]	Speed: 11

Accuracy: 80.70833333333334 %


In [25]:
# save a model to mymodel-symbol.json and mymodel-0100.params
prefix = 'Inception_BN'
epoch = 50
mod.save_checkpoint(prefix, epoch)

INFO:root:Saved checkpoint to "Inception_BN-0050.params"


In [26]:
modelMetric(mod, 'Finish predict of Inception BN..')

INFO:root:Finish predict of Inception BN..
INFO:root:Test accuracy = 0.806823


             precision    recall  f1-score   support

        0.0       0.85      0.79      0.82      2641
        1.0       0.76      0.83      0.79      2137

avg / total       0.81      0.81      0.81      4778

[[2091  550]
 [ 373 1764]]
